In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import(
    StandardScaler)
from sklearn.model_selection import (
    train_test_split)
import statsmodels.formula.api as smf
import pickle
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, SimpleRNN, LSTM, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt

KeyboardInterrupt: 

In [2]:
df=pd.read_csv("data/diabetes_dataset.csv")

df = df.drop(columns='diabetes_stage')
df['gender'] = df['gender'].map({'Male': 1, 'Female': 0, 'Other': 2})
df['ethnicity'] = df['ethnicity'].map({'White' : 0, 'Hispanic' : 1, 'Black' : 2, 'Asian' : 3, 'Other' : 4})
df['education_level'] = df['education_level'].map({'No formal' : 0, 'Highschool' : 1, 'Graduate' : 2, 'Postgraduate' : 3})
df['income_level'] = df['income_level'].map({'Low' : 0, 'Lower-Middle' :1, 'Middle' : 2, 'Upper-Middle' : 3, 'High': 4})
df['employment_status'] = df['employment_status'].map({'Employed' : 0, 'Unemployed' : 1, 'Retired': 2, 'Student':3})
df['smoking_status'] = df['smoking_status'].map({'Never' : 0, 'Former' : 1, 'Current' : 2})
df = df[df['gender'] != 2] #Removing

# Define train and target 
target = df[['diagnosed_diabetes']]
train = df.drop('diagnosed_diabetes', axis = 1)

scaler = StandardScaler()
train = pd.DataFrame(scaler.fit_transform(train), columns=train.columns)

#Division of dataset
x_train_full, x_test = train_test_split(train, test_size=0.2, random_state=123)
y_train_full, y_test = train_test_split(target, test_size=0.2, random_state=123)

x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=0.2)

In [3]:
# Define the activation functions to test
activation_functions = ['relu', 'sigmoid', 'tanh', 'elu', 'selu']

# Function to create a model with a given activation function
def create_model_1(input_dim, activation):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(64, activation=activation),
        Dropout(0.3),
        Dense(32, activation=activation),
        Dropout(0.3),
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])
    return model

def create_model_2(input_dim, activation):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(128, activation=activation),
        Dense(64, activation=activation),
        Dense(32, activation=activation),
        Dense(1, activation='sigmoid')
    ])
    return model

def create_model_3(input_dim, activation):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(32, activation=activation),
        Dense(16, activation=activation),
        Dense(1, activation='sigmoid')
    ])
    return model


# Compile and train the model
def compile_and_train(model, x_train, y_train, x_val, y_val, epochs=20, batch_size=16):
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['recall'])
    with tf.device('/GPU:0'):
        history = model.fit(x_train, y_train,
                        validation_data=(x_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=0)
    return history

# Evaluate the model
def evaluate_model(model, x_test, y_test):
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    y_pred = (model.predict(x_test) > 0.5).astype("int32")
    report = classification_report(y_test, y_pred, output_dict=True)
    return accuracy, report['accuracy'], report['macro avg']['f1-score'], report['macro avg']['recall']

# Example usage
input_dim = x_train.shape[1]
results_1 = {}
results_2 = {}
results_3 = {}

for activation in activation_functions:
    print(f"Testing activation function: {activation}")
    model = create_model_1(input_dim, activation)
    compile_and_train(model, x_train, y_train, x_val, y_val)
    accuracy, precision, f1_score, recall = evaluate_model(model, x_test, y_test)
    results_1[activation] = {'accuracy': accuracy, 'f1_score': f1_score, 'recall' : recall}

for activation in activation_functions:
    print(f"Testing activation function: {activation}")
    model = create_model_2(input_dim, activation)
    compile_and_train(model, x_train, y_train, x_val, y_val)
    accuracy, precision, f1_score, recall = evaluate_model(model, x_test, y_test)
    results_2[activation] = {'accuracy': accuracy, 'f1_score': f1_score, 'recall' : recall}

for activation in activation_functions:
    print(f"Testing activation function: {activation}")
    model = create_model_3(input_dim, activation)
    compile_and_train(model, x_train, y_train, x_val, y_val)
    accuracy, precision, f1_score, recall = evaluate_model(model, x_test, y_test)
    results_3[activation] = {'accuracy': accuracy, 'f1_score': f1_score, 'recall' : recall}

results_SequentialNN_1 = results_1
results_SequentialNN_2 = results_2
results_SequentialNN_3 = results_3

# Save all neural network results to a pickle file
nn_results = {
    'Sequential_Model_1': results_SequentialNN_1,
    'Sequential_Model_2': results_SequentialNN_2,
    'Sequential_Model_3': results_SequentialNN_3
}

with open('neural_network_results_recall_batch32.pkl', 'wb') as f:
    pickle.dump(nn_results, f)

print("Neural network results saved to neural_network_results.pkl")

Testing activation function: relu
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 254us/step
Testing activation function: sigmoid
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 253us/step
Testing activation function: tanh
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 254us/step
Testing activation function: elu
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 285us/step
Testing activation function: selu
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 275us/step
Testing activation function: relu
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 282us/step
Testing activation function: sigmoid
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 293us/step
Testing activation function: tanh
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 288us/step
Testing activation function: elu
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 328us/step
Testing activation function: selu
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 324us/step
Testing activation function: relu
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 264us/step
Testing activation function: sigmoid
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 264us/step
Testing activation function: tanh
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 275

In [4]:
# Define the activation functions to test
activation_functions = ['relu']

batch_sizes = [8, 16, 32, 64]

# Function to create a model with a given activation function
def create_model_1(input_dim, activation):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(64, activation=activation),
        Dropout(0.3),
        Dense(32, activation=activation),
        Dropout(0.3),
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])
    return model

def create_model_2(input_dim, activation):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(128, activation=activation),
        Dense(64, activation=activation),
        Dense(32, activation=activation),
        Dense(1, activation='sigmoid')
    ])
    return model

def create_model_3(input_dim, activation):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(32, activation=activation),
        Dense(16, activation=activation),
        Dense(1, activation='sigmoid')
    ])
    return model


# Compile and train the model
def compile_and_train(model, x_train, y_train, x_val, y_val, epochs=20, batch_size=16):
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['recall'])
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
    with tf.device('/GPU:0'):
        history = model.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[early_stopping],
                    verbose=0)
    return history

# Evaluate the model
def evaluate_model(model, x_test, y_test):
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    y_pred = (model.predict(x_test) > 0.5).astype("int32")
    report = classification_report(y_test, y_pred, output_dict=True)
    return accuracy, report['accuracy'], report['macro avg']['f1-score'], report['macro avg']['recall']

# Example usage
input_dim = x_train.shape[1]
results_1 = {}
results_2 = {}
results_3 = {}

for batch_size in batch_sizes:
    print(f"Testing batch size: {batch_size}")
    model = create_model_1(input_dim, activation)
    compile_and_train(model, x_train, y_train, x_val, y_val, batch_size=batch_size)
    accuracy, precision, f1_score, recall = evaluate_model(model, x_test, y_test)
    results_1[batch_size] = {'accuracy': accuracy, 'f1_score': f1_score, 'recall' : recall}

for batch_size in batch_sizes:
    print(f"Testing batch size: {batch_size}")
    model = create_model_2(input_dim, activation)
    compile_and_train(model, x_train, y_train, x_val, y_val, batch_size=batch_size)
    accuracy, precision, f1_score, recall = evaluate_model(model, x_test, y_test)
    results_2[batch_size] = {'accuracy': accuracy, 'f1_score': f1_score, 'recall' : recall}

for batch_size in batch_sizes:
    print(f"Testing batch size: {batch_size}")
    model = create_model_3(input_dim, activation)
    compile_and_train(model, x_train, y_train, x_val, y_val, batch_size=batch_size)
    accuracy, precision, f1_score, recall = evaluate_model(model, x_test, y_test)
    results_3[batch_size] = {'accuracy': accuracy, 'f1_score': f1_score, 'recall' : recall}


Testing batch size: 8
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 270us/step
Testing batch size: 16
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 271us/step
Testing batch size: 32
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 275us/step
Testing batch size: 64
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step
Testing batch size: 8
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 305us/step
Testing batch size: 16
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 307us/step
Testing batch size: 32
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 322us/step
Testing batch size: 64
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 310us/step
Testing batch size: 8
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 249us/step
Testing batch size: 16
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 248us/step
Testing batch size: 32
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 246us/step
Testing batch size: 64
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 244us/step


In [5]:
# Define the learning rates and loss functions to test
learning_rates = [0.0001, 0.001, 0.01, 0.1]
loss_functions = ['binary_crossentropy', 'hinge']

# Function to create a simple model
def create_model(input_dim):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(128, activation='elu'),
        Dense(64, activation='elu'),
        Dense(32, activation='elu'),
        Dense(1, activation='sigmoid')
    ])
    return model

# Compile and train the model
def compile_and_train(model, x_train, y_train, x_val, y_val, learning_rate, loss, epochs=20, batch_size=16):
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=[tf.keras.metrics.F1Score()])
    with tf.device('/GPU:0'):
        history = model.fit(x_train, y_train,
                        validation_data=(x_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=0)
    return history

# Evaluate the model
def evaluate_model(model, x_test, y_test):
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    y_pred = (model.predict(x_test) > 0.5).astype("int32")
    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    return accuracy, report['accuracy'], report['macro avg']['f1-score']

# Example usage
input_dim = x_train.shape[1]
results = {}

# Test different combinations of learning rates and loss functions
for learning_rate in learning_rates:
    for loss in loss_functions:
        config_name = f"learning_rate={learning_rate},loss={loss}"
        print(f"Testing configuration: {config_name}")
        model = create_model(input_dim)
        compile_and_train(model, x_train, y_train, x_val, y_val, learning_rate, loss)
        accuracy, precision, f1_score = evaluate_model(model, x_test, y_test)
        results[config_name] = {'accuracy': accuracy, 'f1_score': f1_score}

results_LR_Loss = results 

# Save the results to a pickle file
with open('nn_lr_loss_results_scaled_batch16.pkl', 'wb') as f:
    pickle.dump(results_LR_Loss, f)
print("Neural network learning rate and loss function results saved to nn_lr_loss_results.pkl")

Testing configuration: learning_rate=0.0001,loss=binary_crossentropy


KeyboardInterrupt: 

In [ ]:
# Define the activation functions to test
activation_functions = ['relu', 'sigmoid']

# Function to create a model with a given activation function
def create_model_1(input_dim, activation):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(64, activation=activation),
        Dropout(0.3),
        Dense(32, activation=activation),
        Dropout(0.3),
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])
    return model

def create_model_2(input_dim, activation):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(128, activation=activation),
        Dense(64, activation=activation),
        Dense(32, activation=activation),
        Dense(1, activation='sigmoid')
    ])
    return model

def create_model_3(input_dim, activation):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(32, activation=activation),
        Dense(16, activation=activation),
        Dense(1, activation='sigmoid')
    ])
    return model

# Enhanced compile and train function with EarlyStopping
def compile_and_train_with_early_stopping(model, x_train, y_train, x_val, y_val, epochs=50, batch_size=16):
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['recall'])
    
    # Define EarlyStopping callback
    early_stopping = EarlyStopping(
        monitor='val_loss',           # Monitor validation loss
        patience=3,                   # Stop if no improvement for 3 consecutive epochs
        restore_best_weights=True,    # Restore the best weights when stopping
        verbose=1,                    # Print message when stopping
        mode='min',                   # Stop when monitored quantity stops decreasing
        min_delta=0.001               # Minimum change to qualify as improvement
    )
    
    # Train model with callbacks
    with tf.device('/GPU:0'):
        history = model.fit(x_train, y_train,
                        validation_data=(x_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping],
                        verbose=0)
    
    return history, early_stopping.stopped_epoch

# Enhanced evaluate function
def evaluate_model_enhanced(model, x_test, y_test):
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    y_pred = (model.predict(x_test, verbose=0) > 0.5).astype("int32")
    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    return {
        'accuracy': accuracy, 
        'f1_score': report['macro avg']['f1-score'],
        'precision': report['macro avg']['precision'],
        'recall': report['macro avg']['recall']
    }

# Function to plot training history with early stopping indicator
def plot_model_history(history, model_name, activation, stopped_epoch):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot F1 Score (since that's what we're monitoring)
    ax1.plot(history.history['recall'], label='Training Recall', marker='o')
    ax1.plot(history.history['val_recall'], label='Validation Recall', marker='s')
    if stopped_epoch > 0:
        ax1.axvline(x=stopped_epoch, color='red', linestyle='--', 
                   label=f'Early Stop (Epoch {stopped_epoch})')
    ax1.set_title(f'{model_name} - {activation} - Recall')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Recall')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot loss
    ax2.plot(history.history['loss'], label='Training Loss', marker='o')
    ax2.plot(history.history['val_loss'], label='Validation Loss', marker='s')
    if stopped_epoch > 0:
        ax2.axvline(x=stopped_epoch, color='red', linestyle='--',
                   label=f'Early Stop (Epoch {stopped_epoch})')
    ax2.set_title(f'{model_name} - {activation} - Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()

# Training function for each model architecture
def train_model_architecture(model_creator, model_name, input_dim):
    
    results = {}
    training_info = {}
    
    for activation in activation_functions:
        print(f"\n Testing {model_name} with activation: {activation}")
        
        # Create and train model
        model = model_creator(input_dim, activation)
        history, stopped_epoch = compile_and_train_with_early_stopping(
            model, x_train, y_train, x_val, y_val, epochs=50
        )
        
        # Evaluate model
        metrics = evaluate_model_enhanced(model, x_test, y_test)
        
        # Store results
        results[activation] = metrics
        training_info[activation] = {
            'history': history,
            'stopped_epoch': stopped_epoch,
            'total_epochs': len(history.history['loss']),
            'early_stopped': stopped_epoch > 0
        }
        
        # Print results
        print(f"   Accuracy: {metrics['accuracy']:.4f}")
        print(f"   F1-Score: {metrics['f1_score']:.4f}")
        print(f"   Recall: {metrics['recall']:.4f}")
        print(f"   Epochs trained: {len(history.history['loss'])}")
        if stopped_epoch > 0:
            print(f"Early stopped at epoch: {stopped_epoch}")
        else:
            print(f"Completed all epochs")
    
    return results, training_info


# Main execution
input_dim = x_train.shape[1]

# Train all model architectures
print("TRAINING NEURAL NETWORK MODELS WITH EARLY STOPPING")
print("=" * 80)

# Model 1: Simple model with dropout layers
results_1, training_info_1 = train_model_architecture(
    create_model_1, "Model 1 (Simple with Dropout)", input_dim
)

# Model 2: Deeper model with more neurons
results_2, training_info_2 = train_model_architecture(
    create_model_2, "Model 2 (Deeper)", input_dim
)

# Model 3: Smaller model with fewer neurons
results_3, training_info_3 = train_model_architecture(
    create_model_3, "Model 3 (Smaller)", input_dim
)

# Comprehensive results display
print("\n" + "="*100)
print("COMPREHENSIVE RESULTS WITH EARLY STOPPING")
print("="*100)

models_data = [
    ("Model 1 (Simple with Dropout)", results_1, training_info_1),
    ("Model 2 (Deeper)", results_2, training_info_2),
    ("Model 3 (Smaller)", results_3, training_info_3)
]

best_overall = {'model': None, 'activation': None, 'f1_score': 0, 'recall': 0}

for model_name, results, training_info in models_data:
    print(f"\n {model_name}")
    print("-" * 50)
    
    for activation, metrics in results.items():
        info = training_info[activation]
        print(f"   {activation}:")
        print(f"     Accuracy: {metrics['accuracy']:.4f}")
        print(f"     F1-Score: {metrics['f1_score']:.4f}")
        print(f"     Precision: {metrics['precision']:.4f}")
        print(f"     Recall: {metrics['recall']:.4f}")
        print(f"     Epochs: {info['total_epochs']} {'(Early Stopped)' if info['early_stopped'] else '(Full Training)'}")
        
        # Track best overall performance
        if metrics['f1_score'] > best_overall['f1_score']:
            best_overall = {
                'model': model_name,
                'activation': activation,
                'f1_score': metrics['f1_score'],
                'accuracy': metrics['accuracy'],
                'recall': metrics['recall']
            }

# Display best overall model
print(f"\n BEST OVERALL MODEL:")
print(f"   Model: {best_overall['model']}")
print(f"   Activation: {best_overall['activation']}")
print(f"   F1-Score: {best_overall['f1_score']:.4f}")
print(f"   Accuracy: {best_overall['accuracy']:.4f}")
print(f"   Recall: {best_overall['recall']:.4f}")

# Early stopping statistics
print(f"\n EARLY STOPPING STATISTICS:")
total_models = len(activation_functions) * 3
early_stopped_count = 0
total_epochs = 0

for _, _, training_info in models_data:
    for activation, info in training_info.items():
        if info['early_stopped']:
            early_stopped_count += 1
        total_epochs += info['total_epochs']

avg_epochs = total_epochs / total_models
print(f"   Total models trained: {total_models}")
print(f"   Models early stopped: {early_stopped_count}/{total_models} ({early_stopped_count/total_models*100:.1f}%)")
print(f"   Average epochs per model: {avg_epochs:.1f}")

# Plot training history for best model
best_model_data = None
best_training_info = None

with open('nn_early_stopping_results_batch16_recall.pkl', 'wb') as f:
    pickle.dump({
        'models_data': models_data,
        'best_overall': best_overall,
        'early_stopping_stats': {
            'total_models': total_models,
            'early_stopped_count': early_stopped_count,
            'average_epochs': total_epochs / total_models
        }
    }, f)

In [ ]:
feature_ranking = pd.read_csv("datageneral_ranking.csv")


# Get top features based on mutual information score
top_features_mi = feature_ranking.nlargest(10, 'mi_score')['Unnamed: 0'].tolist()
print("Top 10 features based on MI score:")
print(top_features_mi)

# Prepare data with selected features
X_mi = df[top_features_mi].values
y_mi = df['diagnosed_diabetes'].values

scale_mi = StandardScaler()
X_mi = scale_mi.fit_transform(X_mi)

# Split the data
X_train_mi, X_test_mi, y_train_mi, y_test_mi = train_test_split(
    X_mi, y_mi, test_size=0.2, random_state=42
)
X_train_mi, X_val_mi, y_train_mi, y_val_mi = train_test_split(
    X_train_mi, y_train_mi, test_size=0.2, random_state=42
)

print(f"Training set shape: {X_train_mi.shape}")
print(f"Validation set shape: {X_val_mi.shape}")
print(f"Test set shape: {X_test_mi.shape}")

# Define Sequential NN models for MI-selected features
def create_mi_model_1(input_dim, activation='relu'):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(64, activation=activation),
        Dropout(0.3),
        Dense(32, activation=activation),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    return model

def create_mi_model_2(input_dim, activation='relu'):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(128, activation=activation),
        Dense(64, activation=activation),
        Dense(32, activation=activation),
        Dense(1, activation='sigmoid')
    ])
    return model

def create_mi_model_3(input_dim, activation='relu'):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(32, activation=activation),
        Dense(16, activation=activation),
        Dense(1, activation='sigmoid')
    ])
    return model

# Train models with MI-selected features
input_dim_mi = X_train_mi.shape[1]
activation_functions = ['relu', 'sigmoid', 'tanh', 'elu', 'selu']

results_mi_1 = {}
results_mi_2 = {}
results_mi_3 = {}

print("\nTraining Sequential NN models with MI-selected features:")
print("=" * 60)

# Model 1 with MI features
print("\nModel 1 (Simple with Dropout) - MI Features:")
for activation in activation_functions:
    print(f"Testing activation function: {activation}")
    model = create_mi_model_1(input_dim_mi, activation)
    compile_and_train(model, X_train_mi, y_train_mi, X_val_mi, y_val_mi)
    accuracy, precision, f1_score, recall = evaluate_model(model, X_test_mi, y_test_mi)
    results_mi_1[activation] = {'accuracy': accuracy, 'f1_score': f1_score, 'recall': recall}

# Model 2 with MI features
print("\nModel 2 (Deeper) - MI Features:")
for activation in activation_functions:
    print(f"Testing activation function: {activation}")
    model = create_mi_model_2(input_dim_mi, activation)
    compile_and_train(model, X_train_mi, y_train_mi, X_val_mi, y_val_mi)
    accuracy, precision, f1_score, recall = evaluate_model(model, X_test_mi, y_test_mi)
    results_mi_2[activation] = {'accuracy': accuracy, 'f1_score': f1_score, 'recall': recall}

# Model 3 with MI features
print("\nModel 3 (Smaller) - MI Features:")
for activation in activation_functions:
    print(f"Testing activation function: {activation}")
    model = create_mi_model_3(input_dim_mi, activation)
    compile_and_train(model, X_train_mi, y_train_mi, X_val_mi, y_val_mi)
    accuracy, precision, f1_score, recall = evaluate_model(model, X_test_mi, y_test_mi)
    results_mi_3[activation] = {'accuracy': accuracy, 'f1_score': f1_score, 'recall': recall}

# Display results for MI-based models
print("\n" + "="*80)
print("RESULTS FOR MODELS TRAINED WITH MI-SELECTED FEATURES")
print("="*80)

print('\nModel 1 (Simple with dropout) - MI Features:')
for activation, metrics in results_mi_1.items():
    print(f"Activation: {activation}, Accuracy: {metrics['accuracy']:.4f}, F1-Score: {metrics['f1_score']:.4f}, Recall: {metrics['recall']:.4f}")

print('\nModel 2 (Deeper) - MI Features:')
for activation, metrics in results_mi_2.items():
    print(f"Activation: {activation}, Accuracy: {metrics['accuracy']:.4f}, F1-Score: {metrics['f1_score']:.4f}, Recall: {metrics['recall']:.4f}")

print('\nModel 3 (Smaller) - MI Features:')
for activation, metrics in results_mi_3.items():
    print(f"Activation: {activation}, Accuracy: {metrics['accuracy']:.4f}, F1-Score: {metrics['f1_score']:.4f}, Recall: {metrics['recall']:.4f}")

# Find best performing model with MI features
best_mi_model = {'model': None, 'activation': None, 'f1_score': 0, 'accuracy': 0, 'recall': 0}

for model_name, results in [('Model_1_MI', results_mi_1), ('Model_2_MI', results_mi_2), ('Model_3_MI', results_mi_3)]:
    for activation, metrics in results.items():
        if metrics['f1_score'] > best_mi_model['f1_score']:
            best_mi_model = {
                'model': model_name,
                'activation': activation,
                'f1_score': metrics['f1_score'],
                'accuracy': metrics['accuracy'],
                'recall': metrics['recall']
            }

print(f"\nBest MI-based model:")
print(f"Model: {best_mi_model['model']}")
print(f"Activation: {best_mi_model['activation']}")
print(f"F1-Score: {best_mi_model['f1_score']:.4f}")
print(f"Accuracy: {best_mi_model['accuracy']:.4f}")
print(f"Recall: {best_mi_model['recall']:.4f}")

# Save MI-based results
results_MI_NN = {
    'Sequential_MI_Model_1': results_mi_1,
    'Sequential_MI_Model_2': results_mi_2,
    'Sequential_MI_Model_3': results_mi_3,
    'best_model': best_mi_model,
    'selected_features': top_features_mi
}

with open('neural_network_mi_results_scaled_recall_batch16.pkl', 'wb') as f:
    pickle.dump(results_MI_NN, f)

print("\nMI-based neural network results saved to neural_network_mi_results.pkl")

Top 10 features based on MI score:
['hba1c', 'glucose_postprandial', 'glucose_fasting', 'diabetes_risk_score', 'family_history_diabetes', 'age', 'gender', 'systolic_bp', 'ldl_cholesterol', 'physical_activity_minutes_per_week']
Training set shape: (62711, 10)
Validation set shape: (15678, 10)
Test set shape: (19598, 10)

Training Sequential NN models with MI-selected features:

Model 1 (Simple with Dropout) - MI Features:
Testing activation function: relu
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 410us/step
Testing activation function: sigmoid
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 411us/step
Testing activation function: tanh
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 509us/step
Testing activation function: elu
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 425us/step
Testing activation function: selu
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 426us/step

Model 2 (Deeper) - MI Features:
Testing activation function: relu
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step
Testing activation function: sigmoid
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 455us/ste

In [ ]:
# Get top features based on mutual information score
top_features_boruta = feature_ranking.nsmallest(15, 'boruta_rank')['Unnamed: 0'].tolist()
print("Top 10 features based on Boruta score:")
print(top_features_boruta)

# Prepare data with selected features
X_boruta = df[top_features_boruta].values
y_boruta = df['diagnosed_diabetes'].values

scale_boruta = StandardScaler()
X_boruta = scale_boruta.fit_transform(X_boruta)

# Split the data
X_train_boruta, X_test_boruta, y_train_boruta, y_test_boruta = train_test_split(
    X_boruta, y_boruta, test_size=0.2, random_state=42
)
X_train_boruta, X_val_boruta, y_train_boruta, y_val_boruta = train_test_split(
    X_train_boruta, y_train_boruta, test_size=0.2, random_state=42
)

print(f"Training set shape: {X_train_boruta.shape}")
print(f"Validation set shape: {X_val_boruta.shape}")
print(f"Test set shape: {X_test_boruta.shape}")

# Define Sequential NN models for MI-selected features
def create_boruta_model_1(input_dim, activation='relu'):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(64, activation=activation),
        Dropout(0.3),
        Dense(32, activation=activation),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    return model

def create_boruta_model_2(input_dim, activation='relu'):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(128, activation=activation),
        Dense(64, activation=activation),
        Dense(32, activation=activation),
        Dense(1, activation='sigmoid')
    ])
    return model

def create_boruta_model_3(input_dim, activation='relu'):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(32, activation=activation),
        Dense(16, activation=activation),
        Dense(1, activation='sigmoid')
    ])
    return model

# Train models with MI-selected features
input_dim_boruta = X_train_boruta.shape[1]
activation_functions = ['relu', 'sigmoid', 'tanh', 'elu', 'selu']

results_boruta_1 = {}
results_boruta_2 = {}
results_boruta_3 = {}

print("\nTraining Sequential NN models with Boruta-selected features:")
print("=" * 60)

# Model 1 with MI features
print("\nModel 1 (Simple with Dropout) - Boruta Features:")
for activation in activation_functions:
    print(f"Testing activation function: {activation}")
    model = create_boruta_model_1(input_dim_boruta, activation)
    compile_and_train(model, X_train_boruta, y_train_boruta, X_val_boruta, y_val_boruta)
    accuracy, precision, f1_score, recall = evaluate_model(model, X_test_boruta, y_test_boruta)
    results_boruta_1[activation] = {'accuracy': accuracy, 'f1_score': f1_score, 'recall': recall}

# Model 2 with MI features
print("\nModel 2 (Deeper) - MI Features:")
for activation in activation_functions:
    print(f"Testing activation function: {activation}")
    model = create_boruta_model_2(input_dim_boruta, activation)
    compile_and_train(model, X_train_boruta, y_train_boruta, X_val_boruta, y_val_boruta)
    accuracy, precision, f1_score, recall = evaluate_model(model, X_test_boruta, y_test_boruta)
    results_boruta_2[activation] = {'accuracy': accuracy, 'f1_score': f1_score, 'recall': recall}

# Model 3 with MI features
print("\nModel 3 (Smaller) - MI Features:")
for activation in activation_functions:
    print(f"Testing activation function: {activation}")
    model = create_boruta_model_3(input_dim_boruta, activation)
    compile_and_train(model, X_train_boruta, y_train_boruta, X_val_boruta, y_val_boruta)
    accuracy, precision, f1_score, recall = evaluate_model(model, X_test_boruta, y_test_boruta)
    results_boruta_3[activation] = {'accuracy': accuracy, 'f1_score': f1_score, 'recall': recall}

# Display results for MI-based models
print("\n" + "="*80)
print("RESULTS FOR MODELS TRAINED WITH MI-SELECTED FEATURES")
print("="*80)

print('\nModel 1 (Simple with dropout) - MI Features:')
for activation, metrics in results_boruta_1.items():
    print(f"Activation: {activation}, Accuracy: {metrics['accuracy']:.4f}, F1-Score: {metrics['f1_score']:.4f}, Recall: {metrics['recall']:.4f}")

print('\nModel 2 (Deeper) - MI Features:')
for activation, metrics in results_boruta_2.items():
    print(f"Activation: {activation}, Accuracy: {metrics['accuracy']:.4f}, F1-Score: {metrics['f1_score']:.4f}, Recall: {metrics['recall']:.4f}")

print('\nModel 3 (Smaller) - MI Features:')
for activation, metrics in results_boruta_3.items():
    print(f"Activation: {activation}, Accuracy: {metrics['accuracy']:.4f}, F1-Score: {metrics['f1_score']:.4f}, Recall: {metrics['recall']:.4f}")

# Find best performing model with Boruta features
best_boruta_model = {'model': None, 'activation': None, 'f1_score': 0, 'accuracy': 0}

for model_name, results in [('Model_1_Boruta', results_boruta_1), ('Model_2_Boruta', results_boruta_2), ('Model_3_Boruta', results_boruta_3)]:
    for activation, metrics in results.items():
        if metrics['f1_score'] > best_boruta_model['f1_score']:
            best_boruta_model = {
                'model': model_name,
                'activation': activation,
                'f1_score': metrics['f1_score'],
                'accuracy': metrics['accuracy'],
                'recall': metrics['recall']
            }

print(f"\nBest Boruta based model:")
print(f"Model: {best_boruta_model['model']}")
print(f"Activation: {best_boruta_model['activation']}")
print(f"F1-Score: {best_boruta_model['f1_score']:.4f}")
print(f"Accuracy: {best_boruta_model['accuracy']:.4f}")
print(f"Recall: {best_boruta_model['recall']:.4f}")

# Save MI-based results
results_boruta_NN = {
    'Sequential_boruta_Model_1': results_boruta_1,
    'Sequential_boruta_Model_2': results_boruta_2,
    'Sequential_boruta_Model_3': results_boruta_3,
    'best_model': best_boruta_model,
    'selected_features': top_features_boruta}

with open('neural_network_boruta_results_scaled_batch16.pkl', 'wb') as f:
    pickle.dump(results_boruta_NN, f)

print("\nBoruta-based neural network results saved to neural_network_boruta_results.pkl")

In [ ]:
# LSTM Model Architectures
def create_lstm_model_1(input_dim, activation='relu', lstm_units=64):
    """Simple LSTM model with dropout"""
    model = Sequential([
        Input(shape=(input_dim,)),
        Reshape((input_dim, 1)),  # Reshape for LSTM (timesteps, features)
        LSTM(lstm_units, activation=activation, return_sequences=False),
        Dropout(0.3),
        Dense(32, activation=activation),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    return model

def create_lstm_model_2(input_dim, activation='relu', lstm_units=64):
    """Stacked LSTM model"""
    model = Sequential([
        Input(shape=(input_dim,)),
        Reshape((input_dim, 1)),
        LSTM(lstm_units, activation=activation, return_sequences=True),
        Dropout(0.3),
        LSTM(lstm_units//2, activation=activation, return_sequences=False),
        Dropout(0.2),
        Dense(32, activation=activation),
        Dense(1, activation='sigmoid')
    ])
    return model

def create_lstm_model_3(input_dim, activation='relu', lstm_units=32):
    """Bidirectional LSTM model"""
    model = Sequential([
        Input(shape=(input_dim,)),
        Reshape((input_dim, 1)),
        tf.keras.layers.Bidirectional(LSTM(lstm_units, activation=activation)),
        Dropout(0.3),
        Dense(16, activation=activation),
        Dense(1, activation='sigmoid')
    ])
    return model

def create_lstm_model_4(input_dim, activation='relu', lstm_units=48):
    """Deep LSTM with multiple layers"""
    model = Sequential([
        Input(shape=(input_dim,)),
        Reshape((input_dim, 1)),
        LSTM(lstm_units, activation=activation, return_sequences=True),
        Dropout(0.3),
        LSTM(lstm_units//2, activation=activation, return_sequences=True),
        Dropout(0.2),
        LSTM(lstm_units//4, activation=activation, return_sequences=False),
        Dense(16, activation=activation),
        Dense(1, activation='sigmoid')
    ])
    return model

# Enhanced training function with early stopping
def compile_and_train_lstm_with_early_stopping(model, x_train, y_train, x_val, y_val, 
                                              epochs=100, batch_size=16, patience=5):
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['recall']
    )
    
    # Enhanced EarlyStopping for LSTM
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=patience,
        restore_best_weights=True,
        verbose=1,
        mode='min',
        min_delta=0.0001
    )
    
    # Additional callback for learning rate reduction
    lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=patience//2,
        min_lr=0.0001,
        verbose=1
    )
    
    with tf.device('/GPU:0'):
        history = model.fit(
            x_train, y_train,
            validation_data=(x_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stopping, lr_scheduler],
            verbose=0
        )
    
    return history, early_stopping.stopped_epoch

# Enhanced evaluation function
def evaluate_lstm_model(model, x_test, y_test):
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    y_pred = (model.predict(x_test, verbose=0) > 0.5).astype("int32")
    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    
    return {
        'accuracy': accuracy,
        'f1_score': report['macro avg']['f1-score'],
        'precision': report['macro avg']['precision'],
        'recall': report['macro avg']['recall'],
        'loss': loss
    }

# Visualization function for LSTM training
def plot_lstm_history(history, model_name, activation, lstm_units, stopped_epoch):
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    epochs_range = range(1, len(history.history['loss']) + 1)
    
    # Training & Validation Accuracy
    ax1.plot(epochs_range, history.history['recall'], 'bo-', label='Training Accuracy')
    ax1.plot(epochs_range, history.history['val_recall'], 'ro-', label='Validation Accuracy')
    if stopped_epoch > 0:
        ax1.axvline(x=stopped_epoch, color='green', linestyle='--', 
                   label=f'Early Stop (Epoch {stopped_epoch})')
    ax1.set_title(f'{model_name} - {activation} (Units: {lstm_units}) - recall')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Recall')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Training & Validation Loss
    ax2.plot(epochs_range, history.history['loss'], 'bo-', label='Training Loss')
    ax2.plot(epochs_range, history.history['val_loss'], 'ro-', label='Validation Loss')
    if stopped_epoch > 0:
        ax2.axvline(x=stopped_epoch, color='green', linestyle='--',
                   label=f'Early Stop (Epoch {stopped_epoch})')
    ax2.set_title(f'{model_name} - {activation} (Units: {lstm_units}) - Loss')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Learning Rate (if available)
    if 'lr' in history.history:
        ax3.plot(epochs_range, history.history['lr'], 'go-', label='Learning Rate')
        ax3.set_title('Learning Rate Schedule')
        ax3.set_xlabel('Epochs')
        ax3.set_ylabel('Learning Rate')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
    else:
        ax3.text(0.5, 0.5, 'Learning Rate\nNot Recorded', 
                ha='center', va='center', transform=ax3.transAxes)
        ax3.set_title('Learning Rate')
    
    # Training Progress Summary
    final_train_acc = history.history['recall'][-1]
    final_val_acc = history.history['val_recall'][-1]
    final_train_loss = history.history['loss'][-1]
    final_val_loss = history.history['val_loss'][-1]
    
    summary_text = f"""
    Final Training Recall: {final_train_acc:.4f}
    Final Validation Recall: {final_val_acc:.4f}
    Final Training Loss: {final_train_loss:.4f}
    Final Validation Loss: {final_val_loss:.4f}
    Total Epochs: {len(history.history['loss'])}
    Early Stopped: {'Yes' if stopped_epoch > 0 else 'No'}
    """
    
    ax4.text(0.1, 0.5, summary_text, transform=ax4.transAxes, fontsize=12, 
             verticalalignment='center', fontfamily='monospace')
    ax4.set_title('Training Summary')
    ax4.axis('off')
    
    plt.tight_layout()
    plt.show()

# Main LSTM training execution
print(" TRAINING LSTM MODELS WITH BORUTA-SELECTED FEATURES AND EARLY STOPPING")
print("=" * 80)

# Define configurations
activation_functions = ['relu', 'sigmoid']
lstm_units_options = [32, 64, 96]
model_creators = [
    ('LSTM_Model_1_Simple', create_lstm_model_1),
    ('LSTM_Model_2_Stacked', create_lstm_model_2),
    ('LSTM_Model_3_Bidirectional', create_lstm_model_3),
    ('LSTM_Model_4_Deep', create_lstm_model_4)
]

# Initialize results storage
results_lstm= {}
training_histories_lstm = {}
best_lstm_model = {'model': None, 'config': None, 'f1_score': 0, 'accuracy': 0, 'recall': 0}

input_dim = x_train.shape[1]
print(f"Input dimension: {input_dim}")
print(f"Training samples: {x_train.shape[0]}")

# Train all LSTM model configurations
for model_name, model_creator in model_creators:
    print(f"\n  Training {model_name}")
    print("-" * 50)
    
    results_lstm[model_name] = {}
    training_histories_lstm[model_name] = {}
    
    for activation in activation_functions:
        for lstm_units in lstm_units_options:
            config_key = f"{activation}_units_{lstm_units}"
            print(f"  🔧 Configuration: {activation} activation, {lstm_units} LSTM units")
            
            try:
                # Create and train model
                model = model_creator(input_dim, activation, lstm_units)
                history, stopped_epoch = compile_and_train_lstm_with_early_stopping(
                    model, x_train, y_train, x_val, y_val,
                    epochs=100, batch_size=8, patience=7
                )
                
                # Evaluate model
                metrics = evaluate_lstm_model(model, x_test, y_test)
                
                # Store results
                results_lstm[model_name][config_key] = metrics
                training_histories_lstm[model_name][config_key] = {
                    'history': history,
                    'stopped_epoch': stopped_epoch,
                    'total_epochs': len(history.history['loss'])
                }
                
                # Update best model
                if metrics['f1_score'] > best_lstm_model['f1_score']:
                    best_lstm_model = {
                        'model': model_name,
                        'config': config_key,
                        'f1_score': metrics['f1_score'],
                        'accuracy': metrics['accuracy'],
                        'recall': metrics['recall'],
                        'activation': activation,
                        'lstm_units': lstm_units
                    }
                
                # Print results
                print(f"     F1-Score: {metrics['f1_score']:.4f}")
                print(f"     Accuracy: {metrics['accuracy']:.4f}")
                print(f"     Recall: {metrics['recall']:.4f}")
                print(f"     Epochs: {len(history.history['loss'])}")
                if stopped_epoch > 0:
                    print(f"     Early stopped at epoch: {stopped_epoch}")
                
            except Exception as e:
                print(f"     Error: {str(e)}")
                results_lstm[model_name][config_key] = {
                    'accuracy': 0, 'f1_score': 0, 'precision': 0, 'recall': 0, 'loss': float('inf')
                }


# Save LSTM results
results_lstm_complete = {
    'model_results': results_lstm,
    'training_histories': training_histories_lstm,
    'best_model': best_lstm_model,
    'input_dimension': input_dim
}

with open('lstm_results_batch16_recall.pkl', 'wb') as f:
    pickle.dump(results_lstm_complete, f)

print(f"\n LSTM results saved to lstm_results.pkl")


In [ ]:

# Define the activation functions to test
activation_functions = ['relu', 'sigmoid']

# Function to create an RNN model with a given configuration
def create_rnn_model(input_shape, units, activation, num_layers, dropout_rate):
    model = Sequential()
    # Reshape 2D input to 3D for RNN (add timesteps dimension)
    model.add(Input(shape=input_shape))
    model.add(tf.keras.layers.Reshape((input_shape[0], 1)))  # (features, 1) for timesteps
    
    for i in range(num_layers):
        model.add(SimpleRNN(units, activation=activation, return_sequences=(i < num_layers - 1)))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification
    return model

# Compile and train the model
def compile_and_train(model, x_train, y_train, x_val, y_val, epochs=15, batch_size=16):
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=[tf.keras.metrics.F1Score()])
    with tf.device('/GPU:0'):
        history = model.fit(x_train, y_train,
                        validation_data=(x_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=0)
    return history

# Evaluate the model
def evaluate_model(model, x_test, y_test):
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    y_pred = (model.predict(x_test) > 0.5).astype("int32")
    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    return accuracy, report['accuracy'], report['macro avg']['f1-score']

# Make sure you have your data loaded and split before this point
# Example usage (assuming x_train, y_train, x_val, y_val, x_test, y_test are defined)
input_shape = (x_train.shape[1],)  # Note: single dimension tuple
results = {}

#Test different configurations
for activation in activation_functions:
    for units in [16, 32, 64]:
        for num_layers in [1, 2, 3]:
            for dropout_rate in [0.0, 0.3]:
                config_name = f"activation={activation},units={units},layers={num_layers},dropout={dropout_rate}"
                print(f"Testing configuration: {config_name}")
                model = create_rnn_model(input_shape, units, activation, num_layers, dropout_rate)
                compile_and_train(model, x_train, y_train, x_val, y_val)
                accuracy, precision, f1_score = evaluate_model(model, x_test, y_test)
                results[config_name] = {'accuracy': accuracy, 'f1_score': f1_score}

# Display results
print('RNN Model Results:')
for config, metrics in results.items():
   print(f"Config: {config}, Accuracy: {metrics['accuracy']:.4f}, F1-Score: {metrics['f1_score']:.4f}")

# Save RNN results
with open('rnn_results.pkl', 'wb') as f:
    pickle.dump(results, f)
print("RNN results saved to rnn_results.pkl")